# Film Script Analyzer

## Data preparation

## IBM Watson

"IBM Watson is a technology platform that uses natural language processing and machine learning to reveal insights from large amounts of unstructured data"

Watson's interfase needs a [registered account](https://www.ibm.com/account/us-en/signup/register.html?a=MTBmNDg2NDktNDI2MC00&ctx=C001&trial=yes&catalogName=Master&quantity=1&partNumber=WA2PROTRIAL&source=mrsaas-trial-ibmid&pkg=ov49121&S_TACT=000000WB&S_OFF_CD=10000752&siteID=WA&watsonanalytics=true), which assigns a needed username / password.

The process is as follows:
- Authenticate with username/password
- Submit text to be analized
- Receive analysis of text, 30 features that describe the data.

Such features are:
![features](pi_viz.jpg)

Assign variables with username / password.

In [ ]:
iusername = 'ABCDE'
ipassword = '12345'

Define function **'insight'** to submit the queries.

In [ ]:
from watson_developer_cloud import PersonalityInsightsV2 as PerIns

def insight(text,user,password):
    connect = PerIns(username=user,password=password)
    return connect.profile(text)

Send text to analyze, store it in the insights dictionary, then save to disk as a json file.

In [ ]:
insights = {}

for row in df.index:
    if row not in insights.keys():
        temp    = open('scripts/'+row,'r').read()
        insights.update({row:insight(temp,iusername,ipassword)})
        
with open('data/insights.json','w') as f:
    json.dump(insights,f)

Define **'flatten'** function to extract data from the insights dictionary.

In [ ]:
def flatten(orig):
    data = {}
    for c in orig['tree']['children']:
        if 'children' in c:
            for c2 in c['children']:
                if 'children' in c2:
                    for c3 in c2['children']:
                        if 'children' in c3:
                            for c4 in c3['children']:
                                if (c4['category'] == 'personality'):
                                    data[c4['id']] = c4['percentage']
                                    if 'children' not in c3:
                                        if (c3['category'] == 'personality'):
                                                data[c3['id']] = c3['percentage']
    return data

Go through all the insights and flatten them.

In [ ]:
finsights = {}
for key in insights.keys():
    finsights.update({key:flatten(insights[key])})

Convert it to dataframe.

In [ ]:
columns = finsights.values()[0].keys()
index   = df.index
df3     = pd.DataFrame(index=index,columns=columns)

for row in index:
    for feature in columns:
        df3[feature].ix[row] = finsights[row][feature]

From the original **17,271** scrapped files after filtering and cleaning **12,713** remain.

Concatenate the new dataset with the previous one and save it to disk.

This is the core data that will be used later on.

In [ ]:
df0 = pd.concat([df3,df],axis=1)
df0.to_csv('data/df0.csv',encoding='utf-8')